
- The os module has a perfect method to list files in a directory.
- Pandas json normalize could work here but is not necessary to convert the JSON data to a dataframe.
- You may need a nested for-loop to access each sale!
- We've put a lot of time into creating the structure of this repository, and it's a good example for future projects.  In the file functions_variables.py, there is an example function that you can import and use.  If you have any variables, functions or classes that you want to make, they can be put in the functions_variables.py file and imported into a notebook.  Note that only .py files can be imported into a notebook. If you want to import everything from a .py file, you can use the following:
```python
from functions_variables import *
```
If you just import functions_variables, then each object from the file will need to be prepended with "functions_variables"\
Using this .py file will keep your notebooks very organized and make it easier to reuse code between notebooks.

In [2]:
import pandas as pd
import numpy as np
import os
import json
from pprint import pprint

# (this is not an exhaustive list of libraries)
import pandas as pd
import numpy as np
import os
import json
from pprint import pprint
from functions_variables import encode_tags

In [ ]:
# load one file first to see what type of data you're dealing with and what attributes it has

In [3]:
import os

# Specify the directory you want to explore
directory = '/Users/jorgen/Documents/LHL/Supervised-Learning-main/data'

# Use os.listdir to get files
files = os.listdir(directory)

# Print all files
for file in files:
    print(file)

PA_Harrisburg_0.json
LA_BatonRouge_2.json
MO_JeffersonCity_0.json
TN_Nashville_2.json
MD_Annapolis_4.json
NM_SantaFe_3.json
AZ_Phoenix_0.json
KY_Frankfort_4.json
CO_Denver_1.json
FL_Tallahassee_3.json
MI_Lansing_2.json
AR_LittleRock_0.json
ME_Augusta_4.json
GA_Atlanta_4.json
IN_Indianapolis_0.json
MN_St.Paul_4.json
MA_Boston_1.json
WI_Madison_1.json
NE_Lincoln_4.json
AL_Montgomery_4.json
CA_Sacramento_1.json
IA_DesMoines_1.json
AK_Juneau_0.json
OK_OklahomaCity_0.json
NC_Raleigh_0.json
NH_Concord_0.json
DE_Dover_1.json
MT_Helena_1.json
MS_Jackson_0.json
MT_Helena_0.json
MS_Jackson_1.json
RI_Providence_4.json
NH_Concord_1.json
CT_Hartford_4.json
DE_Dover_0.json
NC_Raleigh_1.json
AK_Juneau_1.json
OK_OklahomaCity_1.json
IA_DesMoines_0.json
NY_Albany_4.json
WY_Cheyenne_4.json
CA_Sacramento_0.json
WI_Madison_0.json
MA_Boston_0.json
IL_Springfield_4.json
IN_Indianapolis_1.json
AR_LittleRock_1.json
VT_Montpelier_4.json
CO_Denver_0.json
FL_Tallahassee_2.json
MI_Lansing_3.json
WA_Olympia_4.json


At this point, ensure that you have all sales in a dataframe.
- Is each cell one value, or do some cells have lists?
- Maybe the "tags" will help create some features.
- What are the data types of each column?
- Some sales may not actually include the sale price.  These rows should be dropped.
- Some sales don't include the property type.
- There are a lot of None values.  Should these be dropped or replaced with something?

In [21]:
# Analyze each cell in the DataFrame to check if they contain lists
for column in X_train.columns:
    has_list = X_train[column].apply(lambda x: isinstance(x, list)).any()
    if has_list:
        print(f"Column '{column}' contains a list")

Column 'tags' contains a list
Column 'branding' contains a list
Column 'photos' contains a list
Column 'virtual_tours' contains a list


In [22]:
# If a 'tags' column exists, try printing out unique set of tags
if 'tags' in X_train.columns:
    unique_tags = set(tag for tags in X_train['tags'].dropna() for tag in tags)
    print(f"Unique tags: {unique_tags}")

Unique tags: {'security', 'courtyard_entry', 'clubhouse', 'river_view', 'solar_panels', 'golf_course_lot_or_frontage', 'lake_view', 'hardwood_floors', 'large_kitchen', 'energy_efficient', 'horse_property', 'no_hoa', 'large_porch', 'fireplace', 'den_or_office', 'open_house', 'community_center', 'handicap_access', 'marina', 'community_tennis_court', 'soccer', 'low_hoa', 'shopping', 'community_horse_facilities', 'master_bedroom', 'spa_or_hot_tub', 'greenbelt', 'screen_porch', 'water_view', 'high_ceiling', 'big_lot', 'maintenance', 'equestrian', 'first_floor_master_bedroom', 'farm', 'fenced_yard', 'community_clubhouse', 'ranch', 'medicalcare', 'pets_allowed', 'senior_community', 'kitchen_island', 'guest_house', 'investment_opportunity', 'groundscare', 'white_kitchen', 'guest_parking', 'outbuilding', 'lake', 'mountain_view', 'two_kitchen', 'forced_air', 'park', 'basketball', 'garage_3_or_more', 'ocean_view', 'front_porch', 'indoor_basketball_court', 'gated_community', 'library', 'detached_g

In [23]:
# Print types of each column
print(X_train.dtypes)

status                    int64
data.total                int64
data.count                int64
primary_photo            object
last_update_date         object
source                   object
tags                     object
permalink                object
status                   object
list_date                object
open_houses             float64
description              object
branding                 object
lead_attributes          object
property_id              object
photos                   object
flags                    object
community                object
products                 object
virtual_tours            object
other_listings           object
listing_id               object
price_reduced_amount    float64
matterport               object
0                       float64
street_view_url          object
county                   object
0                       float64
postal_code              object
state                    object
coordinate               object
city    

In [24]:
# Check if a 'sale_price' column exists and if it contains NaN values
if 'sale_price' in X_train.columns:
    print(f"Number of rows before drop: {len(X_train)}")
    X_train = X_train.dropna(subset=['sale_price'])
    print(f"Number of rows after drop: {len(X_train)}")

In [25]:
# Check if a 'property_type' column exists and if it contains NaN values
if 'property_type' in X_train.columns:
    property_type_nan_count = X_train['property_type'].isna().sum()
    print(f"Number of rows without property_type: {property_type_nan_count}")

In [26]:
# Count the number of None values per column
none_count = X_train.isnull().sum()
print(none_count)

status                     0
data.total                 0
data.count                 0
primary_photo            638
last_update_date          50
source                   350
tags                     442
permalink                 24
status                    24
list_date                350
open_houses             6552
description               24
branding                  24
lead_attributes           24
property_id               24
photos                   638
flags                     24
community               6547
products                 411
virtual_tours           5457
other_listings           262
listing_id               350
price_reduced_amount    4564
matterport                24
0                       6552
street_view_url           24
county                    32
0                       6552
postal_code               24
state                     24
coordinate               210
city                      28
state_code                24
line                      37
0             

In [ ]:
# load and concatenate data here
# drop or replace values as necessary

In [9]:
import os
import pandas as pd
import json
from pandas import json_normalize

data_folder = '/Users/jorgen/Documents/LHL/Supervised-Learning-main/data'
json_files = [pos_json for pos_json in os.listdir(data_folder) if pos_json.endswith('.json')]

data_frames = []  # Collate all the DataFrames here

# Load each JSON file and convert it to pandas DataFrame
for file in json_files:
    file_path = os.path.join(data_folder, file)
    with open(file_path, 'r') as json_file:
        json_data = json.load(json_file)
        if isinstance(json_data, dict):
            # Flatten nested dictionary structures if 'data' is a dictionary
            json_df = json_normalize(json_data)
        else:
            # 'data' is not a dict, so it should be a list or scalar
            if isinstance(json_data, list):
                # Flatten list of dictionaries if 'data' is a list
                json_df = json_normalize(json_data)
            else:
                # 'data' is a scalar, so convert it into a DataFrame
                json_df = pd.DataFrame(data=[json_data], columns=['data'])
        data_frames.append(json_df)

# Concatenate all DataFrames
df = pd.concat(data_frames, ignore_index=True)

# Show the first few rows of the DataFrame
print(df.head())

   status  data.total  data.count  \
0     200         648          42   
1     200          48          42   
2     200          21          21   
3     200          18          16   
4     200         442          42   

                                        data.results  
0  [{'primary_photo': {'href': 'https://ap.rdcpix...  
1  [{'primary_photo': {'href': 'https://ap.rdcpix...  
2  [{'primary_photo': {'href': 'https://ap.rdcpix...  
3  [{'primary_photo': {'href': 'https://ap.rdcpix...  
4  [{'primary_photo': {'href': 'https://ap.rdcpix...  


In [10]:
# 'Explode' the column of lists into separate rows
exploded_df = df.explode('data.results')

# Now, 'data.results' is a single dictionary per row,
# so we can convert these dictionaries into separate columns
results_df = exploded_df['data.results'].apply(pd.Series)

# Concatenate the original DataFrame with the new 'results' DataFrame
flattened_df = pd.concat([exploded_df, results_df], axis=1)

# We don't need 'data.results' anymore as its content is now in separate columns
flattened_df = flattened_df.drop(columns=['data.results'])

# Show the first few rows of the new DataFrame
print(flattened_df.head())

  status  data.total  data.count  \
0    200         648          42   
0    200         648          42   
0    200         648          42   
0    200         648          42   
0    200         648          42   

                                       primary_photo      last_update_date  \
0  {'href': 'https://ap.rdcpix.com/8c0b391ae4f9bb...  2024-01-17T00:01:55Z   
0  {'href': 'https://ap.rdcpix.com/40c55dc5d4e29a...  2024-01-16T16:49:36Z   
0  {'href': 'https://ap.rdcpix.com/497a833c1c423b...  2024-01-13T00:03:39Z   
0  {'href': 'https://ap.rdcpix.com/ade15664a2e839...  2024-01-16T07:48:30Z   
0  {'href': 'https://ap.rdcpix.com/a5dada323a7e61...  2024-01-13T00:01:45Z   

                                              source  \
0  {'plan_id': None, 'agents': [{'office_name': N...   
0  {'plan_id': None, 'agents': [{'office_name': N...   
0  {'plan_id': None, 'agents': [{'office_name': N...   
0  {'plan_id': None, 'agents': [{'office_name': N...   
0  {'plan_id': None, 'agents': [{'

In [11]:
# Handle the 'location' column
location_df = flattened_df['location'].apply(pd.Series)

# If 'location' is a nested dictionary, we can flatten it further
address_df = location_df['address'].apply(pd.Series)

# Now, we concatenate the original DataFrame with the 'location' DataFrame and 'address' DataFrame
full_df = pd.concat([flattened_df, location_df, address_df], axis=1)

# Drop the original 'location' and 'address' columns
full_df = full_df.drop(columns=['location', 'address'])

# Show the first few rows
print(full_df.head())

  status  data.total  data.count  \
0    200         648          42   
0    200         648          42   
0    200         648          42   
0    200         648          42   
0    200         648          42   

                                       primary_photo      last_update_date  \
0  {'href': 'https://ap.rdcpix.com/8c0b391ae4f9bb...  2024-01-17T00:01:55Z   
0  {'href': 'https://ap.rdcpix.com/40c55dc5d4e29a...  2024-01-16T16:49:36Z   
0  {'href': 'https://ap.rdcpix.com/497a833c1c423b...  2024-01-13T00:03:39Z   
0  {'href': 'https://ap.rdcpix.com/ade15664a2e839...  2024-01-16T07:48:30Z   
0  {'href': 'https://ap.rdcpix.com/a5dada323a7e61...  2024-01-13T00:01:45Z   

                                              source  \
0  {'plan_id': None, 'agents': [{'office_name': N...   
0  {'plan_id': None, 'agents': [{'office_name': N...   
0  {'plan_id': None, 'agents': [{'office_name': N...   
0  {'plan_id': None, 'agents': [{'office_name': N...   
0  {'plan_id': None, 'agents': [{'

Consider the fact that with tags, there are a lot of categorical variables.
- How many columns would we have if we OHE tags, city and state?
- Perhaps we can get rid of tags that have a low frequency.

In [ ]:
# OHE categorical variables here
# tags will have to be done manually

In [12]:
num_city_cols = full_df['city'].nunique()
num_state_cols = full_df['state'].nunique()

In [16]:
all_tags = set(tag for tags in full_df['tags'] if isinstance(tags, list) for tag in tags)
num_tag_cols = len(all_tags)

tag_freq = {}
for tags_list in full_df['tags']:
    if isinstance(tags_list, list):
        for tag in tags_list:
            if tag not in tag_freq:
                tag_freq[tag] = 0
            tag_freq[tag] += 1

high_freq_tags = {tag for tag, freq in tag_freq.items() if freq > 5}
num_high_freq_tags = len(high_freq_tags)

- Sales will vary drastically between cities and states.  Is there a way to keep information about which city it is without OHE such as using central tendency?
- Could we label encode or ordinal encode?  Yes, but this may have undesirable effects, giving nominal data ordinal values.
- If you replace cities or states with numerical values, make sure that the data is split so that we don't leak data into the training selection. This is a great time to train test split. Compute on the training data, and join these values to the test data
- Drop columns that aren't needed.
- Don't keep the list price because it will be too close to the sale price.

In [ ]:
# perform train test split here
# do something with state and city
# drop any other not needed columns

In [17]:
#Splitting the data into Train and Test

from sklearn.model_selection import train_test_split

X = full_df.drop('list_price', axis=1)  # dropping the 'list_price' column as mentioned
y = full_df['list_price']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [19]:
print(X_train.columns)

Index([              'status',           'data.total',           'data.count',
              'primary_photo',     'last_update_date',               'source',
                       'tags',            'permalink',               'status',
                  'list_date',          'open_houses',          'description',
                   'branding',      'lead_attributes',          'property_id',
                     'photos',                'flags',            'community',
                   'products',        'virtual_tours',       'other_listings',
                 'listing_id', 'price_reduced_amount',           'matterport',
                            0,      'street_view_url',               'county',
                            0,          'postal_code',                'state',
                 'coordinate',                 'city',           'state_code',
                       'line',                      0],
      dtype='object')


In [20]:
#Dealing with 'state' and 'city' columns

df_encoded = pd.get_dummies(X_train, columns=['city', 'state'])

**STRETCH**

- You're not limited to just using the data provided to you. Think/ do some research about other features that might be useful to predict housing prices. 
- Can you import and join this data? Make sure you do any necessary preprocessing and make sure it is joined correctly.
- Example suggestion: could mortgage interest rates in the year of the listing affect the price? 

In [ ]:
# import, join and preprocess new data here

Remember all of the EDA that you've been learning about?  Now is a perfect time for it!
- Look at distributions of numerical variables to see the shape of the data and detect outliers.
- Scatterplots of a numerical variable and the target go a long way to show correlations.
- A heatmap will help detect highly correlated features, and we don't want these.
- Is there any overlap in any of the features? (redundant information, like number of this or that room...)

In [ ]:
# perform EDA here

Now is a great time to scale the data and save it once it's preprocessed.
- You can save it in your data folder, but you may want to make a new `processed/` subfolder to keep it organized